In [1]:
import requests
from bs4 import BeautifulSoup
import wget
import gzip
from tqdm import tqdm
from selectolax.parser import HTMLParser
import os
import pandas as pd
import numpy as np

In [4]:
#parsing 202 urls in 'lst' from tsn.ua/robots.txt... thats contains ~800K urls of all site.

URL = 'https://tsn.ua/ru/sitemap_index.xml' #url from \robots.txt
r = requests.get(URL)
bsObj = BeautifulSoup(r.content)

lst_of_xml_gz = []

for item in bsObj.find_all('loc'): # parsing all URLs of sitemap from sitemap_index.xml
    lst_of_xml_gz.append(item.get_text())

lst_of_xml_gz[0:10]

['https://tsn.ua/sitemap/full/ru/sitemap_all_1.xml.gz',
 'https://tsn.ua/sitemap/full/ru/sitemap_all_2.xml.gz',
 'https://tsn.ua/sitemap/full/ru/sitemap_all_3.xml.gz',
 'https://tsn.ua/sitemap/full/ru/sitemap_all_4.xml.gz',
 'https://tsn.ua/sitemap/full/ru/sitemap_all_5.xml.gz',
 'https://tsn.ua/sitemap/full/ru/sitemap_all_6.xml.gz',
 'https://tsn.ua/sitemap/full/ru/sitemap_all_7.xml.gz',
 'https://tsn.ua/sitemap/full/ru/sitemap_all_8.xml.gz',
 'https://tsn.ua/sitemap/full/ru/sitemap_all_9.xml.gz',
 'https://tsn.ua/sitemap/full/ru/sitemap_all_10.xml.gz']

In [3]:
#creating new folder in project 'data_xml_gz1' for 202 xml files.gzip
os.mkdir('C:\\Users\\T\\data_tsn\\data_xml_gz')

In [5]:
#downloading 202 xml.gz file from robots.txt that contains ~800K URLs
path = 'C:\\Users\\T\\data_tsn\\data_xml_gz'
for url in lst_of_xml_gz:
    wget.download(url, out = path)

In [24]:
#checking downloaded files
files = os.listdir(r'C:\\Users\\T\\data_tsn\\data_xml_gz')
for item in files[0:10]:
    print(item)

sitemap_all_1.xml.gz
sitemap_all_10.xml.gz
sitemap_all_100.xml.gz
sitemap_all_101.xml.gz
sitemap_all_102.xml.gz
sitemap_all_103.xml.gz
sitemap_all_104.xml.gz
sitemap_all_105.xml.gz
sitemap_all_106.xml.gz
sitemap_all_107.xml.gz


In [4]:
#parsing all URLs from xml.gz local files
lst_of_URLs = []
for item in tqdm(files):
    path = f'C:\\Users\\T\\data_tsn\\data_xml_gz\{item}'
    f=gzip.open(path, 'rb')
    file_content = f.read()
    tree = HTMLParser(file_content)
    node = tree.css('url')
    for item in node:
        lst_of_URLs.append(item.text().split())

In [7]:
#checking downloaded files
print(lst_of_URLs[0:5])
print(len(lst_of_URLs))

[['https://tsn.ua/ru/den-v-istorii-d/20-iyulya-v-istorii-mezhdunarodnyy-den-torta-nachalo-istorii-bmw-i-den-ledenca-na-palochke-99780.html', '2021-07-20', 'daily', '0.7'], ['https://tsn.ua/ru/svit/21-iyulya-zvezdy-rodivshiesya-segodnya-99788.html', '2021-07-21', 'daily', '0.7'], ['https://tsn.ua/ru/svit/21-iyulya-v-istorii-pervyy-chelovek-na-lune-i-izdanie-posledney-knigi-potteriany-138838.html', '2021-07-21', 'daily', '0.7'], ['https://tsn.ua/ru/den-v-istorii-d/1-avgusta-v-istorii-vozvraschenie-uolta-disneya-iz-evropy-brizhit-bardo-iz-meksiki-i-pobeda-mayka-taysona-139768.html', '2021-08-01', 'daily', '0.7'], ['https://tsn.ua/online', '2022-02-23', 'daily', '1']]
804310


In [8]:
# deleting all samples that have len != 4
for item in lst_of_URLs:
    if len(item) != 4:
        lst_of_URLs.remove(item)
# 8 samples lost, cause wrong data parsing or bad xml html in tsn.ua
len(lst_of_URLs)

804302

In [10]:
#for downloading all .html from lst_of_URLs we must create 
#new folder for urls.txt x20 files with all URLs separeted from list

os.mkdir('C:\\Users\\T\\data_tsn\\data_txt')

In [18]:
#creating 20 urls.txt with 40216 URLs in one files
#804302 ---> 804320 for make divide to 20 
for i in range(20):
    with open(f'C:/Users/T/data_tsn/data_txt/urls{i}.txt', 'w') as out:
        for item in lst_of_URLs[40216 * i:40216 * (i+1)-1]:
            out.write(item[0] + '\n')


In [19]:
#checking created files:
files = os.listdir(r'C:\\Users\\T\\data_tsn\\data_txt')
for item in files:
    print(item)

.ipynb_checkpoints
urls0.txt
urls1.txt
urls10.txt
urls11.txt
urls12.txt
urls13.txt
urls14.txt
urls15.txt
urls16.txt
urls17.txt
urls18.txt
urls19.txt
urls2.txt
urls3.txt
urls4.txt
urls5.txt
urls6.txt
urls7.txt
urls8.txt
urls9.txt


In [14]:
#creating new folder in project for downloaded .html files from URLs of data_txt .txt files
os.mkdir('E:\\data_html')

In [26]:
#downloading all html links from urls 0 - 20 .txt in local disk for future parsing.
for i in range(0,20):
    os.system(f'START /B /HIGH wget --compression=gzip -P E:/data_html -i C:/Users/T/data_tsn/data_txt/urls{i}.txt')


In [27]:
#from downloaded html files deleting all that not have .html and not good
files = os.listdir(r'E:\data_html')
pattern = r'[A-Za-z0-9._%+-]+html'
for item in files:
    match = re.fullmatch(pattern, item)
    if match is not None:
        pass    
    else:
        os.remove(f"E:/data_html/{item}")
        
#all index.html have nothing inside trying to remove it
try:
    os.remove(f"E:/data_html/index.html") 
except:
    print('None index')

In [3]:

#parsing all good html to result_list
files = os.listdir(r'E:\data_html')
result_of_parsing = []
for item in tqdm(files):
    dict_for_elem = {} 
    # Opening the html file
    HTMLFile = open(f'E:/data_html/{item}', "r", encoding='utf-8')

    # Reading the file
    index = HTMLFile.read()

    # Creating a BeautifulSoup object and specifying the parser
    tree = HTMLParser(index)



    #get tag in header
    try:
        dict_for_elem['header_tag'] = tree.css_first('dd[class=c-bar__label]').text()
    except AttributeError as e:
        print('header_tag')
        print(item)


    # лінк статті
    try:
        for item in tree.css('a[class=sr-only]'):
            dict_for_elem['href'] = item.attributes['href']
    except AttributeError as e:
        print('href')
        print(item)


    # get title

    try:
        for item in tree.css('h1[class=c-card__title]'):
            dict_for_elem['title'] = item.text()
    except AttributeError as e:
        print('title')
        print(item)

    #get time_date
    try:
        dict_for_elem['time_date'] = tree.css_first('time').text()
    except AttributeError as e:
        print('time_date')
        print(item)


    #get view
    try:
        dict_for_elem['view'] = tree.css('footer[class=c-card__foot]')[0].text()
    except IndexError as e:
        print('view')
        print(item)


    #get bold_h1
    try:
        items = tree.css('div[class=c-article__lead]')
        for item in items:
            dict_for_elem['bold_h1'] = item.text()
    except AttributeError as e:
        print('bold_h1')
        print(item)

    #get text
    try:
        item = tree.css('div[class=c-article__body]')
        for item in item:
            dict_for_elem['text'] = item.text()
    except AttributeError as e:
        print('text')
        print(item)

    #get tags
    try:
        lst_tag = []
        for item in tree.css('a[class=c-tag__box]'):
            lst_tag.append(item.text())
        dict_for_elem['tags'] = lst_tag
    except AttributeError as e:
        print('tags')
        print(item)
    result_of_parsing.append(dict_for_elem)

In [17]:
df = pd.DataFrame(result_of_parsing)

In [14]:
df.to_csv(r'C:\Users\T\data_tsn\parse.csv', mode='a', index=False, header=False)

CLEANING parser.csv

In [2]:
tsn = pd.read_csv('parse.csv')

In [10]:
tsn['title'].sample(5)

22448                       \nКогда начинается рекрутинг?\n
429989    \nСеть умилил американский полицейский, которы...
588855    \n"Завел в сарай и издевался". На Житомирщине ...
438429    \nСмертельно больным американцам разрешили при...
430138    \nСеверная Корея обвинила США в провокациях пе...
Name: title, dtype: object

In [11]:
#cleaning title
def len_of_title():
    len_befor = tsn['title'].str.len()
    tsn['title'] = tsn['title'].str.replace('\n','')
    tsn['title'] = tsn['title'].str.replace('\xa0',' ')
    tsn['title'] = tsn['title'].str.replace('\t',' ')
    tsn['title'] = tsn['title'].str.replace('&quot;','')
    tsn['title'] = tsn['title'].str.replace(r'\s{2,}', ' ', regex=True)
    tsn['title'] = tsn['title'].str.strip(' ')
    tsn['title'][tsn['title'].str.isnumeric().fillna(False)] = np.nan # 1034 row with numbers is converted to np.nan
    tsn['title'][tsn['title'].str.contains(r'[\d]+_', regex=True).fillna(False)] = np.nan # 952 row with 01_mfofsa 10_asmkdma... numbers and some text
    len_after = tsn['title'].str.len()
    return len_befor - len_after
result_of_title = len_of_title()
result_of_title.value_counts() #checking how much len deleted for see some Errors

2.0     458078
0.0     326817
3.0       1915
1.0       1007
12.0       137
4.0         54
5.0         25
24.0        17
6.0          4
13.0         2
7.0          1
14.0         1
15.0         1
Name: title, dtype: int64

In [13]:
tsn['title'].sample(5)

232265    Цены на лекарства в оккупированном Крыму вырос...
25176     Либкин и Барбир приготовили французский суп "Б...
581120    Янукович пророчит Украине прорыв в космической...
604569    Арсен Аваков заявил о пророссийской информацио...
8075      Чиновники смогут подать е-декларации только в ...
Name: title, dtype: object

In [14]:
tsn['time_date'].sample(5)

56372     10:56, 09.03.16
6126      01:49, 11.08.16
657935    20:35, 08.05.18
259902    17:21, 06.03.15
415777    15:51, 20.05.22
Name: time_date, dtype: object

In [15]:
#time and date cleaning and setting time_date to index
time_date_columns = ['time', 'date']
tsn[time_date_columns] = tsn['time_date'].str.split(', ', expand=True)
tsn = tsn.drop(labels='time_date', axis='columns')
date_columns = ['day', 'month', 'year']
tsn[date_columns] = tsn['date'].str.split('.', expand=True)
tsn = tsn.drop(labels='date', axis='columns')
tsn['time_date'] = tsn['time'] + ', ' + tsn['day'] + '-' + tsn['month'] + '-' + '20' + tsn['year']
tsn = tsn.drop(labels=['time','day','month','year',], axis='columns')
tsn['time_date'] = pd.to_datetime(tsn['time_date'], format="%H:%M, %d-%m-%Y")
tsn = tsn.set_index('time_date').sort_index()

In [16]:
tsn.head(5)

,header_tag,href,title,view,bold_h1,text,tags
time_date,,,,,,,
2007-05-26 00:00:00,Видео,https://tsn.ua/ru/video/video-novini/test01.html,test01,"Дата публикации00:00, 26.05.07Просмотры\n ...",NaN,NaN,['\n test01\n ']
2007-05-26 00:00:00,Видео,https://tsn.ua/ru/video/video-novini/dog.html,dog,"Дата публикации00:00, 26.05.07Просмотры\n ...",NaN,NaN,['\n dog\n ']
2007-05-26 00:00:00,Видео,https://tsn.ua/ru/video/video-novini/tree.html,tree,"Дата публикации00:00, 26.05.07Просмотры\n ...",NaN,NaN,['\n tree\n ']
2007-05-27 18:35:00,Видео,https://tsn.ua/ru/video/video-novini/cherkasi-...,Черкаси без газу,"Дата публикации18:35, 27.05.07Просмотры\n ...",NaN,NaN,[]
2007-05-28 12:00:00,Видео,https://tsn.ua/ru/video/video-novini/tch-25-05...,TCH 25 05 2007,"Дата публикации12:00, 28.05.07Просмотры\n ...",NaN,NaN,['\n tch25052007\n ...


In [19]:
tsn['bold_h1'].sample(5)

time_date
2012-09-10 10:44:00    Ждать, пока глупые станут мудрее - жизни не хв...
2014-03-18 17:51:00    Канал Russia Today на YouTube был заблокирован...
2019-06-07 03:58:00    Африканский Союз приостановил членство этой ст...
2012-04-17 18:53:00    Кого выбрала Даша Астафьева в свою группу, про...
2022-07-27 07:15:00    Начались 154 сутки героического противостояния...
Name: bold_h1, dtype: object

In [20]:
#Cleaning bold_h1
def len_of_bold_h1():
    len_befor = tsn['bold_h1'].str.len()
    tsn['bold_h1'] = tsn['bold_h1'].str.replace('\n','')
    tsn['bold_h1'] = tsn['bold_h1'].str.replace('\xa0','')
    tsn['bold_h1'] = tsn['bold_h1'].str.replace('\t',' ')
    tsn['bold_h1'] = tsn['bold_h1'].str.replace(r'\s{2,}', ' ', regex=True)
    tsn['bold_h1'] = tsn['bold_h1'].str.strip(' ')
    len_after = tsn['bold_h1'].str.len()
    return len_befor - len_after
result_of_bold_h1 = len_of_bold_h1()
result_of_bold_h1.value_counts() #checking how much len deleted for see some Errors

0.0     635007
2.0      49299
1.0      42361
4.0      13853
3.0       3675
5.0        653
6.0        401
7.0         98
10.0        73
8.0         69
9.0         29
11.0        16
14.0        11
12.0        10
20.0         9
13.0         7
16.0         6
28.0         6
22.0         6
15.0         5
18.0         5
27.0         3
21.0         3
30.0         2
17.0         2
29.0         2
26.0         2
19.0         2
24.0         2
36.0         1
78.0         1
45.0         1
25.0         1
34.0         1
33.0         1
31.0         1
Name: bold_h1, dtype: int64

In [21]:
tsn['bold_h1'].sample(5)

time_date
2015-06-11 13:30:00      Противник продолжает бить на всех направлениях.
2015-02-28 17:00:00    Организацию похорон взял на себя зять покойног...
2015-01-20 13:36:00               Боевые действия в зоне АТО не утихают.
2014-09-24 11:44:00         Злоумышленник неоднократно выезжал в Москву.
2019-08-14 16:42:00    Решать вопросы водоснабжения нельзя в отрыве о...
Name: bold_h1, dtype: object

In [15]:
for item in tsn['tags'].sample(5):
    print(item)

['\n                        Выпуски ТСН\n        ', '\n                        коронавирус в украине\n        ', '\n                        гость\n        ', '\n                        игорь кузин\n        ']
['\n                        голос страны\n        ', '\n                        сельсовет\n        ', '\n                        тьома паучек\n        ', '\n                        паучек\n        ', '\n                        дима монатик\n        ']
['\n                \xa0\n                        россия\n        ']
['\n                        Киев\n        ', '\n                        стрельба\n        ', '\n                        преступники\n        ', '\n                        Toyota\n        ', '\n                        правоохранители\n        ']
['\n                        арест\n        ', '\n                        Черное море\n        ', '\n                        капитан\n        ', '\n                        катер\n        ', '\n                        джубга\n 

In [16]:
#cleaning tags
tsn['tags'] = tsn['tags'].str.lower()
tsn['tags'] = tsn['tags'].replace(regex={r"(\['\\n\s+)":"['",r'\\n\s+':'',r'\\xa0':''})
tsn['tags'] = tsn['tags'].apply(eval)

In [17]:
for item in tsn['tags'].sample(5):
    print(item)

['адвокат', 'савченко', 'фейгин', 'грушники']
['шендерович']
['запорожье', 'мошенники', 'мошенник', 'волонтеры', 'благотворительный']
['кения', 'спасение животных', 'мощные наводнения']
['ребенок', 'дом', 'донетчина', 'снежное']


In [18]:
for item in tsn['view'].sample(5):
    print(item)

Дата публикации10:20, 12.03.19Просмотры
        12
    Поделиться:FacebookTwitterWhatsAppTelegramViberСкопировать ссылку
Дата публикации15:05, 23.12.13Просмотры
        1863
    Поделиться:FacebookTwitterWhatsAppTelegramViberСкопировать ссылку
Дата публикации12:58, 23.11.14Просмотры
        3830
    Поделиться:FacebookTwitterWhatsAppTelegramViberСкопировать ссылку
Дата публикации01:30, 26.11.17Просмотры
        4222
    Поделиться:FacebookTwitterWhatsAppTelegramViberСкопировать ссылку
Дата публикации17:52, 15.02.18Просмотры
        8
    Поделиться:FacebookTwitterWhatsAppTelegramViberСкопировать ссылку


In [19]:
tsn['view'] = tsn['view'].str.extract(r'(\b[0-9]+[k-к-К]?[m-М-M]?\s)',expand=True)
tsn['view'] = tsn['view'].str.replace('\n','')
tsn['view'] = tsn['view'].str.replace('к','000')
tsn['view'] = tsn['view'].str.replace('М','000000')
tsn['view'] = tsn['view'].fillna('0')
tsn['view'] = tsn['view'].astype('int32')

In [34]:
tsn['view'].sample(5)

time_date
2018-08-18 01:17:00     3435
2020-07-29 08:55:00        6
2016-07-07 08:09:00        3
2016-06-29 22:47:00    10000
2022-03-06 08:58:00     3653
Name: view, dtype: int32

In [22]:
for item in tsn['text'].sample(5):
    print(item, end='\n\n')
    print('separator', end='\n\n')

К обстрелам из тяжелой артиллерии наши морпехи уже привыкли. Не могут привыкнуть к смертям собратьев. Уходят из жизни совсем юные 20-летние контрактники. Они вместе с опытными старшими воинами защищают позиции под Мариуполем. Их рубеж - Водяное. И за этот рубеж они отдают свои жизни.

separator

Конструктивная схема здания: монолитный железобетонный каркас. Применяются системы навесных вентилируемых фасадов, с утеплением стен минеральной ватой.Соинвестором строительства жилищно-офисного комплекса "Новопечерська Вежа", который возводится в Печерском районе Киева - по пер. Кленовому, 7, 7-А, 7-Б; ул. Киквидзе, 41-А, 41-43, стала ведущая немецкая архитектурно-строительная компания "Möser GmbH".

По словам руководителя компании-соинвестора и заказчика строительства ООО "Гранд Луол" Александра Семинищева, киевляне знакомы с этим комплексом под названием ЖК "Панорама". Но, пересмотрев качественные и имиджевые характеристики жилого комплекса после прихода иностранного инвестора, было принято 

In [22]:
def len_of_text():
    len_befor = tsn['text'].str.len()
    tsn['text'] = tsn['text'].str.replace(r'(Также читайте\s+.+\s+)', '', regex=True)
    tsn['text'] = tsn['text'].str.replace(r'(Также читайте.+)', '', regex=True)
    tsn['text'] = tsn['text'].str.replace(r'(Читайте также[\D\d]+)', '', regex=True)
    tsn['text'] = tsn['text'].str.replace(r'(\n\b.{0,40}читайте также[\d\D]+)', '', regex=True)
    tsn['text'] = tsn['text'].str.replace('[\n\r]', '', regex=True)
    tsn['text'] = tsn['text'].str.replace(r'(\s{2,})', '',regex=True)
    len_after = tsn['text'].str.len()
    return len_befor - len_after
result_of_text = len_of_text()

In [23]:
for item in tsn['text'].sample(5):
    print(item, end='\n\n')
    print('separator', end='\n\n')

Уже совсем скоро американцы выберут себе нового президента. В честь будущих первых супругов страны уже изготавливают специальные украшения и значки. Комплектов будет два - для Хиллари и Билла и Мелании и Дональда.


Прогноз погоды на 16 апреля.Сегодня, 16 апреля, в западных и восточных регионах Украины ожидаются дожди.Как сообщает Укргидрометцентр, на севере, в центре и юге страны - преимущественно без осадков. Температура воздуха днем составит 14-22 градуса тепла.На западе Украины синоптики прогнозируют температуру15-22 тепла, на севере – 15-18 тепла, на востоке – 14-18 градусов, на юге – 16-21 тепла, а в центре - от 16 до 18 градусов.В Киеве сегодня ожидается солнечная погода, температура воздуха 15-17 градусов тепла.


Работы по строительству планируется завершить до 2019 года.Латвия построила на границе с Россией 23 километры стены с целью предотвратить попытки нелегальной иммиграции.Об этом сообщил пресс-секретарь Государственной пограничной охраны Латвии Евгения Позняк, передает 